# tensorflow 强化学习入门

## 使用环境：

ubuntu-20.04LTS

cuda 11.1

nvidia 460.73

python 3.7.10

tensorflow 2.5.0rc1


In [1]:
import tensorflow as tf
tf_V = tf.__version__
print("version of tensorflow is :  ", tf_V)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
version of tensorflow is :   2.5.0-rc1


**检验是否可以使用GPU**

test if the gpu is available

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

有些时候nvidia会自动更新显卡驱动并导致bug，此时会报错：“Failed to initialize NVML: Driver/library version mismatc”， 并且此时在tensorflow中无法使用GPU（上述操作输出为“False”）

解决方案：sudo reboot （重启电脑）

## tensorflow 教程资源

tf 自2.4版本之后在api中提供了直接调用DQN、DDPG等网络的函数，十分方便

tensorflow官方也放出了有关教程：

1、所有教程
https://tensorflow.google.cn/tutorials

2.1、actor-critic架构
https://tensorflow.google.cn/tutorials/reinforcement_learning/actor_critic
    相关notebook 下载：
    https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/reinforcement_learning/actor_critic.ipynb
    目前已经在该目录下
    
2.2、tf-agent 用法
https://tensorflow.google.cn/agents
    相关notebook 下载：
    https://github.com/tensorflow/agents
    目前已经在本目录下


## 强化学习简介
> 引用自tensorflow强化学习官方文档 0_intro_rl

> Reinforcement learning (RL) is a general framework where agents learn to perform actions in an environment so as to maximize a reward. The two main components are the environment, which represents the problem to be solved, and the agent, which represents the learning algorithm.

强化学习（Reinforcement learning，RL）是一个通用框架，在该框架中智能体（agent）可以与环境通过输出动作（action）进行交互并最大化奖励（reward）。其中我们可以看到，在这个框架中一共有两个主要组成部分：1、环境（environment），即智能体要解决的问题。2、智能体（agent），即解决问题的学习算法。

> The agent and environment continuously interact with each other. At each time step, the agent takes an action on the environment based on its *policy* $\pi(a_t|s_t)$, where $s_t$ is the current observation from the environment, and receives a reward $r_{t+1}$ and the next observation $s_{t+1}$ from the environment. The goal is to improve the policy so as to maximize the sum of rewards (return).

智能体和环境之间会持续地相互交互。在训练中的每一个步骤（step）中，智能体都会通过**策略（*policy*）（$\pi(a_t|s_t)$**生成一个对应的动作（action）（$a_t$），其中$s_t$是当前从环境观测到的状态（observation），并且智能体在每一次和环境的交互之后都会接收到一个奖励（reward）（$r_{t+1}$）和下一个步骤的状态（$s{t+1}$）。总体的目标是最大化奖励（reward）

> Note: It is important to distinguish between the `state` of the environment and the `observation`, which is the part of the environment `state` that the agent can see, e.g. in a poker game, the environment state consists of the cards belonging to all the players and the community cards, but the agent can observe only its own cards and a few community cards. In most literature, these terms are used interchangeably and observation is also denoted as $s$.

需要注意的是：需要区分状态（state）和观测（observation）这两个概念。观测（observation）是指整个环境里面所有的状态，状态（state）只是观测（observation）的一部分，即智能体所能接触到的信息。比如说在一个扑克游戏里。state指的是agent只能看得到的自己的纸牌，observation是所有玩家的扑克牌

> This is a very general framework and can model a variety of sequential decision making problems such as games, robotics etc. 

强化学习是一个泛化性很强的框架，可以适用于各种连续决策问题（sequential decision making problems），例如游戏、机器人等等。

## 举例：倒立摆问题（CartPole）
> The Cartpole environment is one of the most well known classic reinforcement learning problems ( the *"Hello, World!"* of RL). A pole is attached to a cart, which can move along a frictionless track. The pole starts upright and the goal is to prevent it from falling over by controlling the cart.  

倒立摆问题（CartPole）相当于是强化学习中的“hello world”问题。在该问题中，一根杆子连接到一个只能沿着一条直线移动到滑块上。杆子在初始时刻是垂直的，需要调整滑块的运动来保证杆子不再倒下。

> *   The observation from the environment $s_t$ is a 4D vector representing the position and velocity of the cart, and the angle and angular velocity of the pole. 
> *   The agent can control the system by taking one of 2 actions $a_t$: push the cart right (+1) or left (-1). 
> *   A reward $r_{t+1} = 1$ is provided for every timestep that the pole remains upright. The episode ends when one of the following is true:
>  * the pole tips over some angle limit
>  * the cart moves outside of the world edges
>  * 200 time steps pass. 

*  对环境的观测（$s_t$）是一个四维矢量，其中分别是滑块的位置和速度以及杆子的角度和角速度。
*  智能体的动作（$a_t$）只有一个维度，而且取离散值（+1，-1），其中+1代表向右推动滑块；-1代表向左推动滑块  The agent can control the system by taking one of 2 actions $a_t$: push the cart right (+1) or left (-1). 
*  如果在第$t$个时刻（step），杆子依然没有倒下，那么就给一个正的reward：$r_{t+1}=1$，当满足以下几个条件中的任意一个时一个回合（episode）结束。
  * 当杆子偏移角度超出一个阈值时；
  * 当滑块移动出边界时
  * 超出时间限制时（200个step）
  
 智能体的最终目的是学习到一个策略，即$\pi (a_t|s_t)$来最大化一个回合内的奖励：$\sum_{t=0}^T \gamma^t r_t$，其中$\gamma$是针对未来的奖励的衰减系数（范围在[0,1]之间）。This parameter helps us focus the policy, making it care more about obtaining rewards quickly.
 
 ## DQN 算法
 
 > The  [DQN (Deep Q-Network) algorithm](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) was developed by DeepMind in 2015. It was able to solve a wide range of Atari games (some to superhuman level) by combining reinforcement learning and deep neural networks at scale. The algorithm was developed by enhancing a classic RL algorithm called Q-Learning with deep neural networks and a technique called *experience replay*.
 
[深度Q网络算法（DQN，Deep Q-Network）](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)在2015年被DeepMind团队研发。通过将强化学习和深度神经网络相结合，它能够解决各种Atari游戏（甚至在有些游戏中的表现超过人类）。 
![雅达利（Atari）游戏机](https://cdn.jsdelivr.net/gh/Brook1711/fig_for_blog/img/20210428134916.png)

### Q学习（Q-Learning）

> Q-Learning is based on the notion of a Q-function. The Q-function (a.k.a the state-action value function) of a policy $\pi$, $Q^{\pi}(s, a)$, measures the expected return or discounted sum of rewards obtained from state $s$ by taking action $a$ first and following policy $\pi$ thereafter. We define the optimal Q-function $Q^*(s, a)$ as the maximum return that can be obtained starting from observation $s$, taking action $a$ and following the optimal policy thereafter. The optimal Q-function obeys the following *Bellman* optimality equation: 

Q学习（Q-Learning）是基于Q函数的概念。所谓的Q函数也就是状态-动作价值函数（state-action value function），用于衡量一个策略$\pi$中的状态动作的价值，写作$Q^{\pi}(s, a)$。它的含义是